In [1]:
import json
from kafka import KafkaConsumer
import pymssql
import pandas as pd

from elasticsearch import Elasticsearch
from elasticsearch.helpers import scan
from elasticsearch.helpers import bulk
from elasticsearch.helpers import BulkIndexError

from cryptography.fernet import Fernet # generar Fernet key
import configparser # Parsear el fichero de configuración

from datetime import datetime, timedelta, date


import warnings

# Deshabilitar warnings para que no aparezcan al confiar en el certificado
warnings.filterwarnings('ignore')

In [2]:
# Generar key con el nombre de fichero dado
def generate_encryption_key(dest_file_path:str):    
    # Generate a random encryption key
    key = Fernet.generate_key()

    # Save the encryption key to a file (you should protect this file securely)
    with open(dest_file_path, 'wb') as key_file:
        key_file.write(key)

def encrypt_password_with_key(password:str, 
                              key_file_path:str, 
                              config_file_path:str, 
                              connection_name:str,
                              parameter_name="password"):
    # Cargar el ficheo con laclave
    with open(key_file_path, 'rb') as key_file:
        key = key_file.read()

    # Crear un cifrado Fernet con la clave
    cipher_suite = Fernet(key)

    # Contraseña a encriptar
    password_to_encrypt = password.encode()  # Convertir a bytes

    # Encriptar contraseña
    encrypted_password = cipher_suite.encrypt(password_to_encrypt)

    # Actualizar fichero de configuración
    config = configparser.ConfigParser()
    config.read(config_file_path)  # Cargar el ficer de configuración

    # Almacenar contraseña encriptada en el fichero de configuración
    config.set(connection_name, parameter_name, encrypted_password.decode())
    # Otra forma de hacerlo
    #config[connection_name][parameter_name] = encrypted_password.decode()  # Convert to a string

    # Guardar configuración nueva
    with open(config_file_path, 'w') as config_file:
        config.write(config_file)


def decrypt_password_with_key(key_file_path:str, 
                              config_file_path:str, 
                              connection_name:str,
                              parameter_name="password"):
    # Cargar el ficheo con la clave
    with open(key_file_path, 'rb') as key_file:
        key = key_file.read()

    # Crear un cifrado Fernet con la clave
    cipher_suite = Fernet(key)
    
    # Leer la contraseña encriptada del fichero de configuración
    config = configparser.ConfigParser()
    config.read(config_file_path)  # Cargar el ficer de configuración
    
    encrypted_password = config.get(connection_name, parameter_name).encode()

    # Desencriptar contraseña
    decrypted_password = cipher_suite.decrypt(encrypted_password).decode()  # Convertir a string
    return decrypted_password


def connect_to_elastic(key_file_path:str,
                       config_file_path:str, 
                       connection_name:str,
                       parameter_name='password'):
    
    # Leer el fichero de configuración
    config = configparser.ConfigParser()
    config_file_path = config_file_path
    config.read(config_file_path)  # Adjust the path to your config file

    # Almacenar información de la conexión
    connection_name = 'elastic'
    connection_config = config[connection_name]
    elastic_host = connection_config['host']
    elastic_port = connection_config['port']
    elastic_user = connection_config['user']
    elastic_password = connection_config['password'] # Contraseña encriptada

    # Desncriptar contraseña
    decrypted_password = decrypt_password_with_key(
        key_file_path=key_file_path,
        config_file_path=config_file_path,
        connection_name=connection_name,
        parameter_name=parameter_name
    )
    
    elastic_password = decrypted_password
    elastic_endpoint = f'{elastic_host}:{elastic_port}'
    elastic_url = f'https://{elastic_endpoint}'

    # Crear conexión
    es = Elasticsearch(
        elastic_url,
        http_auth=(elastic_user, elastic_password),
        verify_certs=False
    )

    return es

def run_elastic_query(elastic_connection, query, index_name):
    page_size = 10000
    scroll_id = None
    
    # Ejecutar query
    response = scan(
            elastic_connection,
            query=query,
            index=index_name,
            scroll="6m",
            size=page_size
    )
    
    # Generar resultado
    results = []
    for hit in response:
        results.append(hit["_source"])

    return results      

In [3]:
from elasticsearch import Elasticsearch

# Elasticsearch URL and authentication details
elastic_url = 'https://10.210.4.118:9200'
elastic_user = 'elastic'
elastic_password = 'eilv2Dh04gbCBYAouc1o'

# Instantiate Elasticsearch with the trusted SSL context
es = Elasticsearch(
    elastic_url,
    http_auth=(elastic_user, elastic_password),
    verify_certs=False
)

# Procesar la fecha

In [4]:
fecha_hora_str = "Sun Oct 29 2023 10:55:31 GMT+0100"

# Formato de la cadena de fecha y hora
formato = "%a %b %d %Y %H:%M:%S GMT%z"

# Convertir la cadena a objeto datetime
fecha_hora_objeto = datetime.strptime(fecha_hora_str, formato)

# Imprimir la fecha y hora en el formato deseado
fecha_hora_sin_zona_horaria = fecha_hora_objeto.replace(tzinfo=None)
print("Fecha y hora convertidas:", fecha_hora_sin_zona_horaria)


Fecha y hora convertidas: 2023-10-29 10:55:31


## Obtener periodo actual

In [5]:
# Definición de variables
index_name = "raw-asm"
#datetime_ini = date.today() - timedelta(days=7)
datetime_ini = datetime.now() - timedelta(days=1)

#datetime_ini = "04-10-2022"

datetime_ini = str(datetime_ini) 
#datetime_ini = datetime.strptime(datetime_ini, "%Y-%m-%d %H:%M:%S")
# Convertir la cadena a objeto datetime
datetime_ini = datetime.strptime(datetime_ini, "%Y-%m-%d %H:%M:%S.%f")
# Eliminar los decimales de los segundos
datetime_ini_str = datetime_ini.strftime("%Y-%m-%d %H:%M:%S")
#datetime_ini = int(datetime_ini.timestamp()) * 1000 
print("datetime_ini", datetime_ini_str)

datetime_fi = datetime.now()
#datetime_fi = "05-10-2022"

datetime_fi = str(datetime_fi)
datetime_fi = datetime.strptime(datetime_fi,"%Y-%m-%d %H:%M:%S.%f")
datetime_fi_str = datetime_fi.strftime("%Y-%m-%d %H:%M:%S")
#datetime_fi = int(datetime_fi.timestamp()) * 1000
print("datetime_fi", datetime_fi_str)



datetime_ini 2023-12-01 12:35:07
datetime_fi 2023-12-02 12:35:07


## Obtener periodo anterior

In [6]:
interval_days = (datetime_fi - datetime_ini).days
print("interval_days", interval_days)
previous_datetime_ini =  datetime_ini - timedelta(days=interval_days)
previous_datetime_fi = datetime_fi - timedelta(days=interval_days)

interval_days 1


# Crear la query para la fecha seleecionada

In [7]:
# Creación de la query
query = {
    "query": {
        "bool": {
            "filter": [
                # Añadir filtro por entorno o tags
                #{
                #    "term": {
                #        "sensorType.keyword": {
                #            "value": sensor_type
                #        }
                #    }
                #},
                {
                    "range": {
                        "fechaProcesado": {
                            "gte": datetime_ini,
                            "lte": datetime_fi,
                        }
                    }
                }
            ]
        }
    }
}
print(query)

{'query': {'bool': {'filter': [{'range': {'fechaProcesado': {'gte': datetime.datetime(2023, 12, 1, 12, 35, 7, 834269), 'lte': datetime.datetime(2023, 12, 2, 12, 35, 7, 835404)}}}]}}}


# Ejecutar query

In [8]:
index_name="raw-asm"
actual_data = run_elastic_query(es, query, index_name)
actual_data_df = pd.DataFrame(actual_data)
actual_data_df.head()

,type,idTest,tiempoPaso,idError,fecha,idGrupTest,idPaso,fechaProcesado,nombreTest,idRobot,nombreRobot,id,harFile,errorMsg,urlFailed,pathError,elementError,errorSel
0,Result,263,0.602,0,20231201161511,292,17,2023-12-01 16:15:11,BETA_HBLO-02_Fitxers_Firefox,22,RobotSeleniumSilkBPI,20231201161511_17_22_263,NaN,NaN,NaN,NaN,NaN,NaN
1,Result,263,0.609,0,20231201161516,292,19,2023-12-01 16:15:16,BETA_HBLO-02_Fitxers_Firefox,22,RobotSeleniumSilkBPI,20231201161516_19_22_263,NaN,NaN,NaN,NaN,NaN,NaN
2,Result,145,7.96,0,20231201161412,10,1,2023-12-01 16:14:12,HBFIL-08_M2PE_Consultas_y_Altas,38,RobotSeleniumSilk38,20231201161412_1_38_145,NaN,NaN,NaN,NaN,NaN,NaN
3,Result,99341,2.273,0,20231201161520,15,4,2023-12-01 16:15:20,LANPROVA_CAPABILITIES_HBAQX-01_Comptes_Emprese...,9992,RobotSeleniumCapabilities,20231201161520_4_9992_99341,20231201161530_99341.har,NaN,NaN,NaN,NaN,NaN
4,GroupedResult,394,15.667,1,20231201161525,684,9999,2023-12-01 16:15:25,BETA_HBLO-32_BKGE4_RCCBOX_Chrome,41,RobotSeleniumSilk41,20231201161525_9999_41_394,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
actual_data_df
actual_data_df[actual_data_df['type'] == "GroupedResult"]['errorMsg']

'4         NaN\n12        NaN\n19        NaN\n31        NaN\n37        NaN\n         ... \n235094    NaN\n235107    NaN\n235119    NaN\n235120    NaN\n235123    NaN\nName: errorMsg, Length: 34076, dtype: object'

# Crear la query para la fecha anterior

In [9]:
# Creación de la query
query = {
    "query": {
        "bool": {
            "filter": [
                # Añadir filtro por entorno o tags
                #{
                #    "term": {
                #        "sensorType.keyword": {
                #            "value": sensor_type
                #        }
                #    }
                #},
                {
                    "range": {
                        "fechaProcesado": {
                            "gte": previous_datetime_ini,
                            "lte": previous_datetime_fi,
                        }
                    }
                }
            ]
        }
    }
}
print(query)

{'query': {'bool': {'filter': [{'range': {'fechaProcesado': {'gte': datetime.datetime(2023, 11, 30, 12, 35, 7, 834269), 'lte': datetime.datetime(2023, 12, 1, 12, 35, 7, 835404)}}}]}}}


# Ejecutar query

In [10]:
previous_data = run_elastic_query(es, query, index_name)
previous_data_df = pd.DataFrame(previous_data)
previous_data_df.head()

,type,idGrupTest,idTest,idRobot,nombreTest,idError,idPaso,tiempoPaso,fechaProcesado,nombreRobot,fecha,id,harFile,errorMsg,urlFailed,pathError,errorSel,elementError
0,Result,452,188,29,BETA_HBLO-18_Targetes_Empresa_Firefox,0,2,1.405,2023-11-30 18:16:11,RobotSeleniumSilk29,20231130181611,20231130181611_2_29_188,NaN,NaN,NaN,NaN,NaN,NaN
1,Result,452,188,29,BETA_HBLO-18_Targetes_Empresa_Firefox,0,5,3.429,2023-11-30 18:16:40,RobotSeleniumSilk29,20231130181640,20231130181640_5_29_188,NaN,NaN,NaN,NaN,NaN,NaN
2,GroupedResult,452,188,29,BETA_HBLO-18_Targetes_Empresa_Firefox,1,9999,6.464,2023-11-30 18:16:40,RobotSeleniumSilk29,20231130181640,20231130181640_9999_29_188,NaN,NaN,NaN,NaN,NaN,NaN
3,Result,15,216,35,HBPOR-05_Pagament_de_impostos_i_rebuts,0,1,0.104,2023-11-30 18:16:15,RobotSeleniumSilk35,20231130181615,20231130181615_1_35_216,NaN,NaN,NaN,NaN,NaN,NaN
4,Result,2585,647,10,BETA_HBAQX-02_Comptes_Particulars_cat_Edge,0,3,0.679,2023-11-30 18:16:37,RobotSeleniumSilk10,20231130181637,20231130181637_3_10_647,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
xxx

NameError: name 'xxx' is not defined

# Preprocesar los datos para añadir tags y entornos

In [ ]:
import re

def split_camelcase_string(s):
    return re.findall(r'[A-Z](?:[a-z]+|[A-Z]*(?=[A-Z]|$))', s)

In [ ]:
actual_data_df['tags'] = actual_data_df.apply(lambda x: split_camelcase_string(x['nombreTest']) + split_camelcase_string(x['nombreRobot']), axis=1)
actual_data_df['entorno'] = actual_data_df['nombreRobot'].apply(lambda x: split_camelcase_string(x))[1][1]

previous_data_df['tags'] = previous_data_df.apply(lambda x: split_camelcase_string(x['nombreTest']) + split_camelcase_string(x['nombreRobot']), axis=1)
previous_data_df['entorno'] = previous_data_df['nombreRobot'].apply(lambda x: split_camelcase_string(x))[1][1]

In [ ]:
 previous_data_df.apply(lambda x: split_camelcase_string(x['nombreTest']) + split_camelcase_string(x['nombreRobot']), axis=1)

In [ ]:
actual_data_df.head()

# Gráfico 1

In [ ]:
# calcular número total de ejecuciones y la comparación en el periodo previo
total_actual_tours = len(actual_data_df)
total_previous_tours = len(previous_data_df)
print("total_actual_tours", total_actual_tours)
print("total_previous_tours", total_previous_tours)

# calcular número total de errores y la comparación en el periodo previo
filtered_actual_data_df = actual_data_df[actual_data_df['errorMsg'].notnull() & (actual_data_df['errorMsg'] != '')]
total_actual_errors = len(filtered_actual_data_df)

filtered_previous_data_df = previous_data_df[previous_data_df['errorMsg'].notnull() & (previous_data_df['errorMsg'] != '')]
total_previous_errors = len(filtered_previous_data_df)
print("total_actual_errors", total_actual_errors)
print("total_previous_errors", total_previous_errors)

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()


fig.add_trace(go.Indicator(
    value = total_actual_tours,
    delta = {'reference': total_previous_tours },
    title = {'text': "Tours totales "},
    gauge = {
        'axis': {'visible': False,
                'range': [None, 100]}},
    domain = {'row': 0, 'column': 0}))


fig.add_trace(go.Indicator(
    value = total_actual_errors,
    delta = {'reference': total_previous_errors},
    title = {'text': "Tours con errores"},
    gauge = {
        'bar': {'color': "lightpink"},
        'axis': {'visible': False}},
    domain = {'row': 0, 'column': 1}))
'''
fig.add_trace(go.Indicator(
    value = 120,
    gauge = {
        'shape': "bullet",
        'axis' : {'visible': False}},
    domain = {'x': [0.05, 0.5], 'y': [0.15, 0.35]}))

fig.add_trace(go.Indicator(
    mode = "number+delta",
    value = 300,
    domain = {'row': 0, 'column': 1}))

fig.add_trace(go.Indicator(
    mode = "delta",
    value = 40,
    domain = {'row': 1, 'column': 1}))
'''
fig.update_layout(
    grid = {'rows': 1, 'columns': 2, 'pattern': "independent"},
    template = {'data' : {'indicator': [{
        'title': {'text': "Speed"},
        'mode' : "number+delta+gauge",
        'delta' : {'reference': 90}}]
                         }})

# Gráfico 2: Diagrama de barras de los 5 tours con más errores por paso

- Eje X: Nombre de los 5 tours con más errores
- Eje Y: Número de errores por cada paso

In [ ]:

top_5_tours_df = filtered_actual_data_df.groupby(['nombreTest']).size().reset_index(name='Count').sort_values(by='Count', ascending=False)
top_5_tours_list = top_5_tours_df.head(5)['nombreTest'].tolist()
filtered_actual_top_5_df = filtered_actual_data_df[filtered_actual_data_df['nombreTest'].isin(top_5_tours_list)]
filtered_actual_top_5_df.head()

grouped_actual_top_5_df = filtered_actual_top_5_df.groupby(['nombreTest', 'idPaso']).size().reset_index(name='Count').sort_values(by='idPaso', ascending=True)



In [ ]:
min_step = int(grouped_actual_top_5_df['idPaso'].unique().min())
max_step = int(grouped_actual_top_5_df['idPaso'].unique().max())
tours_list = grouped_actual_top_5_df['nombreTest'].unique().tolist()
bar_plot_list = []

for i in range (min_step, max_step+ 1, 1):
    i = str(i)
    result_array = []
    for test in tours_list:
        if any((grouped_actual_top_5_df['nombreTest'] == test) & (grouped_actual_top_5_df['idPaso'] == i)):
            result_array.append(grouped_actual_top_5_df.loc[(grouped_actual_top_5_df['nombreTest'] == test) & (grouped_actual_top_5_df['idPaso'] == i), 'Count'].values[0])
        else:
            result_array.append(0)
    dict = {'name': f'Paso {i}', 'data':result_array }
    bar_plot_list.append(dict)

In [ ]:
print(tours_list )
bar_plot_list

# Gráfico 3: Diagrama de barras con los mensajes de error más comunes

- Eje X: Nombre de los 5 errorMsg más comunes
- Eje Y: Número de errores por cada errorMsg

In [ ]:
top_5_errormsg_df = filtered_actual_data_df.groupby(['errorMsg']).size().reset_index(name='Count').sort_values(by='Count', ascending=False)
top_5_errormsg_list = top_5_errormsg_df.head(5)['errorMsg'].tolist()
top_5_count_list = {'name':'Errores', 'data': top_5_errormsg_df.head(5)['Count'].tolist()}

print(top_5_count_list)
top_5_errormsg_list

# Gráfico 4: Series temporales de la evolución de los errores sobre el top 5 de tours

- Eje X: Tiempo
- Eje Y: Nº Errores

Si el intervalo de tiempo es menor a 3 días, graficar por horas; si es mayor o igual a tres días, grficar por día



In [ ]:
interval_days
top_5_tours_list
31 * 12

In [ ]:
filtered_actual_data_df['fechaProcesado']= pd.to_datetime(filtered_actual_data_df['fechaProcesado'])

if interval_days < 3:
    filtered_actual_data_df['fecha_nueva'] = filtered_actual_data_df['fechaProcesado'].dt.strftime('%d-%m-%Y %H')
elif interval_days < 31:
    filtered_actual_data_df['fecha_nueva'] = filtered_actual_data_df['fechaProcesado'].dt.strftime('%d-%m-%Y')
elif interval_days < 372:
    filtered_actual_data_df['fecha_nueva'] = filtered_actual_data_df['fechaProcesado'].dt.strftime('%m-%Y')
else:
    filtered_actual_data_df['fecha_nueva'] = filtered_actual_data_df['fechaProcesado'].dt.strftime('%Y')
    
grouped_dates_top_5_df = filtered_actual_data_df.groupby(['nombreTest', 'fecha_nueva']).size().reset_index(name='Count').sort_values(by='fecha_nueva', ascending=True)

grouped_dates_top_5_df = grouped_dates_top_5_df[grouped_dates_top_5_df['nombreTest'].isin(top_5_tours_list)]
dates_list = grouped_dates_top_5_df['fecha_nueva'].unique().tolist()

line_plot_list = []
for tour in top_5_tours_list:
    result_array = []
    for date in dates_list:
        check_df = (grouped_dates_top_5_df['fecha_nueva'] == date) & (grouped_dates_top_5_df['nombreTest'] == tour)
        if any(check_df):
            result_array.append(grouped_dates_top_5_df.loc[check_df, 'Count'].values[0])
        else:
            result_array.append(0)
    dict = {'name': tour, 'data':result_array }
    line_plot_list.append(dict)


In [ ]:
line_plot_list
dates_list

# Gráfico 5: Tags más comúmes sobre los tours con errores



In [ ]:
tags_df = filtered_actual_data_df.explode('tags').groupby(['tags']).size().reset_index(name='Count').sort_values(by='Count', ascending=False)
top_8_tags_df  = tags_df.head(8)
top_8_tags_list = top_8_tags_df['tags'].tolist()
top_8_tags_count_list = top_8_tags_df['Count'].tolist()
top_8_tags_count_list


In [ ]:
top_5_tags_list = list(top_5_errormsg_df.head(5)['errorMsg'])
top_5_tags_list = {'name':'Errores', 'data': list(top_5_errormsg_df.head(5)['Count'])}

print(top_5_count_list)
top_5_errormsg_list

In [ ]:
check_df = (grouped_dates_top_5_df['fecha_nueva'] == '28-10-2023 19') & (grouped_dates_top_5_df['nombreTest'] == 'LANPROVA_BETA_HBLO-34_MyBox_Jubilacion_Chrome')

grouped_dates_top_5_df.loc[check_df, 'Count'].values[0]

In [ ]:
line_plot_list = []
for tour in top_5_tours_list:
    result_array = []
    for date in dates_list:
        check_df = (grouped_dates_top_5_df['fecha_nueva'] == date) & (grouped_dates_top_5_df['nombreTest'] == tour)
        if any(check_df):
            result_array.append(grouped_dates_top_5_df.loc[check_df, 'Count'].values[0])
        else:
            result_array.append(0)
    dict = {'name': tour, 'data':result_array }
    line_plot_list.append(dict)
line_plot_list

In [ ]:
filtered_actual_top_5_df

In [ ]:
# Filtrar los registros que tienen un valor para 'errorMsg'
filtered_df = data_df[data_df['errorMsg'].notnull() & (data_df['errorMsg'] != '')]

# Contar el número de registros para cada entorno
count_by_env = filtered_df.groupby('entorno').size()

# Convertir los resultados de la agrupación a una lista
categories = count_by_env.index.tolist()
values = count_by_env.tolist()

# Configurar el ángulo de las barras en el gráfico polar
theta = np.linspace(0.0, 2 * np.pi, len(values), endpoint=False)

# Ancho de cada barra en el gráfico polar
width = (2 * np.pi) / len(values)

# Crear un gráfico polar
ax = plt.subplot(111, projection='polar')
colors = ['skyblue', 'salmon', 'lightgreen', 'gold']

bars = ax.bar(theta, values, width=width, bottom=0.0, align='center', alpha=0.5, color=colors)

# Personalizar el gráfico polar
ax.set_xticks(theta)
ax.set_xticklabels(categories)

# Añadir un título
plt.title("Número de errores por entorno")

# Mostrar el gráfico
plt.show()



# Grafico 4: Diagrama de barras de los 5 tours con más errores por paso
- El nombre del tour en el eje X
- El nñumero de errores en el eje y  

In [ ]:
top_5_tours_list

In [ ]:
filtered_df['fechaProcesado'].unique()
filtered_df['fechaProcesado']= pd.to_datetime(filtered_df['fechaProcesado'])

filtered_df['fecha_nueva'] = filtered_df['fechaProcesado'].dt.strftime('%Y-%m-%d')

data_df['fechaProcesado']= pd.to_datetime(data_df['fechaProcesado'])
data_df['fecha_nueva'] = data_df['fechaProcesado'].dt.strftime('%Y-%m-%d')



In [ ]:
# Agrupar el DataFrame por fecha y contar el número de errores
count_by_date = filtered_df.groupby('fecha_nueva').size()
#count_by_date = filtered_df.groupby('fecha_nueva').size()

# Crear un gráfico de líneas
plt.plot(count_by_date.index, count_by_date.values, marker='o', linestyle='-')
#plt.plot(count_by_date.index, count_by_date.values, marker='o', linestyle='-')

# Añadir etiquetas y título al gráfico
plt.xlabel('Fecha')
plt.ylabel('Número de errores')
plt.title('Evolución del número de errores por fecha')

# Rotar las etiquetas del eje x para una mejor visualización
plt.xticks(rotation=45)

# Mostrar el gráfico
plt.show()

In [ ]:
# Crear el gráfico de líneas usando Plotly
fig = go.Figure()

# Agregar el primer conjunto de datos
fig.add_trace(go.Scatter(x=count_by_date.index, y=count_by_date.values, mode='lines+markers', name='DataFrame 1'))

# Agregar el segundo conjunto de datos
#fig.add_trace(go.Scatter(x=df2['fecha'], y=df2['valor'], mode='lines+markers', name='DataFrame 2'))

# Actualizar el diseño del gráfico
fig.update_layout(title='Evolución del número de errores por fecha',
                  xaxis_title='Fecha',
                  yaxis_title='Valor')

# Mostrar el gráfico
fig.show()

In [ ]:
# Crear el gráfico de líneas utilizando Plotly para cada entorno
fig = go.Figure()

# Iterar sobre los entornos únicos y agregar una línea para cada entorno
for entorno in filtered_df['entorno'].unique():
    df_entorno = filtered_df[filtered_df['entorno'] == entorno]
    count_by_date = df_entorno.groupby('fecha_nueva').size()

    fig.add_trace(go.Scatter(x=count_by_date.index, y=count_by_date.values, mode='lines+markers', name=entorno))

# Actualizar el diseño del gráfico
fig.update_layout(title='Evolución del número de errores por entorno',
                  xaxis_title='Fecha',
                  yaxis_title='Valor')

# Mostrar el gráfico
fig.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Example data
categories = ['Category A', 'Category B', 'Category C', 'Category D']
count_by_env = [25, 40, 30, 45]

# Setting the angles for the categories
theta = np.linspace(0.0, 2 * np.pi, len(values), endpoint=False)

# Width of each bar in the polar plot
width = (2*np.pi) / len(values)

# Creating a polar plot
ax = plt.subplot(111, projection='polar')
bars = ax.bar(theta, values, width=width, bottom=0.0, align='center', alpha=0.5)

# Customizing the polar plot
ax.set_xticks(theta)
ax.set_xticklabels(categories)

# Adding a title
plt.title("Radial Bar Chart Example")

# Showing the plot
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Example data
categories = ['Category A', 'Category B', 'Category C', 'Category D']
values = count_by_env

# Setting the angle of the bars
theta = np.linspace(0.0, 2 * np.pi, len(values), endpoint=False)

# Creating the polar plot
ax = plt.subplot(111, polar=True)
bars = ax.bar(theta, values, width=0.4, align='center', alpha=0.5)

# Customizing the radial bars
for bar, category in zip(bars, categories):
    bar.set_facecolor(plt.cm.jet(np.random.rand()))
    bar.set_label(category)

# Adding a title and legend
plt.title("Radial Bar Chart Example")
plt.legend(bars, categories, loc='upper right')

# Showing the plot
plt.show()


In [ ]:
import plotly.graph_objects as go

# Datos de ejemplo
labels = ['A', 'B', 'C', 'D', 'E']
values = [15, 30, 45, 10, 25]

# Crear el gráfico de sectores por la mitad
fig = go.Figure(data=[go.Pie(labels=labels, values=values, direction='clockwise', hole=0.4)])

# Ajustar el diseño del gráfico
fig.update_layout(title_text="Gráfico de sectores por la mitad")

# Mostrar el gráfico
fig.show()

In [ ]:
kafka_topic = "asm-preprocessed-data"
consumer = KafkaConsumer(kafka_topic, bootstrap_servers=["10.210.4.117:9092","10.210.4.118:9092","10.210.4.119:9092"],
                         auto_offset_reset='earliest', enable_auto_commit=True,
                         auto_commit_interval_ms=1000, group_id='asm-preprocessed-data-1')

In [ ]:
for message in consumer:
        #try: 
        message_data = json.loads(message.value.decode('utf-8'))
        print(message_data)
        #except:
        #    print("----error", message.timestamp)


In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch.helpers import scan
from elasticsearch.helpers import bulk
from elasticsearch.helpers import BulkIndexError

from datetime import datetime, timedelta, date